# Import packages

In [5]:
import numpy as np
from scipy.signal import fftconvolve
from scipy.io.wavfile import write
from scipy.io import wavfile
import matplotlib.pyplot as plt

import pyroomacoustics as pra
from pyroomacoustics.doa import circ_dist
import torchaudio
import torch
from IPython.display import Audio

In [6]:
import IPython.display as ipd
import wave

import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
#import torch.
from torch.utils.data import Dataset
from torchvision import transforms
from torchsummary import summary
import torch.nn.functional as F
import math

from scipy.io import wavfile as wav
import pandas as pd
import torchaudio
import soundfile as sf

import array
import math
import numpy as np
import random
import wave

from torch.utils.data.sampler import SubsetRandomSampler
import pickle

In [7]:
def get_at():
    angle_table_base = np.array([
        120.0,
        60.0,
        0.0,
        300.0,
        240.0,
        180.0]
    )

    at = angle_table_base
    at = np.sort(at) 
    return at


def get_label_index(angle, at):
    label_index = 0
    angle = int(angle)
    if (angle < at[1] and angle >= at[0]):
        label_index = 0
        angle = angle / 60
    elif (angle < at[2] and angle >= at[1]):
        label_index = 1
        angle = (angle - at[1]) / 60
    elif (angle < at[3] and angle >= at[2]):
        label_index = 2
        angle = (angle - at[2]) / 60
    elif (angle < at[4] and angle >= at[3]):
        label_index = 3
        angle = (angle - at[3]) / 60
    elif (angle < at[5] and angle >= at[4]):
        label_index = 4
        angle = (angle - at[4]) / 60
    elif (angle < 360 and angle >= at[5]):
        label_index = 5
        angle = (angle - at[5]) / 60
        
    if at[label_index] == angle:
        angle = 0
    if angle == 360:
        angle = 0
    return label_index, angle

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import transforms
from torchsummary import summary
import torch.nn.functional as F
import torchaudio
from torch.utils.data.sampler import SubsetRandomSampler
import os
import datetime
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import utils.utils_base as uBase
from models.KSL_EFFI_v3 import EfficientNet
from dataset_MT_KWS import MC_MT_KWS_Dataset, Overlap_interference
from IPython.display import Audio
from dataset_MT_newData import new_Dataset

annotation = "newData3.csv"

train_annotation = 'newData3_train.csv'
valid_annotation = 'newData3_valid.csv'
test_annotation = 'newData3_test.csv'

train_dataset = new_Dataset(annotations_file=train_annotation)
valid_dataset = new_Dataset(annotations_file=valid_annotation)
test_dataset = new_Dataset(annotations_file=test_annotation)


batch_size = 32
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size)
valid_dataloader = torch.utils.data.DataLoader(test_dataset,
                                               batch_size=1)

net = EfficientNet()
path = "weight/20230523-221newData/" + str(35) + ".pt"
net.load_state_dict(torch.load(path))
net.eval()

EfficientNet(
  (_conv_stem): Conv1dSamePadding(6, 32, kernel_size=(3,), stride=(2,), bias=False)
  (_bn0): BatchNorm1d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv1dSamePadding(32, 32, kernel_size=(3,), stride=(1,), groups=32, bias=False)
      (_bn1): BatchNorm1d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (_se_reduce): Conv1dSamePadding(32, 8, kernel_size=(1,), stride=(1,))
      (_se_expand): Conv1dSamePadding(8, 32, kernel_size=(1,), stride=(1,))
      (_project_conv): Conv1dSamePadding(32, 16, kernel_size=(1,), stride=(1,), bias=False)
      (_bn2): BatchNorm1d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): MBConvBlock(
      (_expand_conv): Conv1dSamePadding(16, 96, kernel_size=(1,), stride=(1,), bias=False)
      (_bn0): BatchNorm1d(96, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (_depthwi

In [8]:

def DOA(region_y, angle_y, region_p, angle_p, threshold, class_y, class_p):
    at = get_at()
    doa_err = 0.0
    doa_acc = 0
    residual = 0
    bo = 0
    count = 0
    sig = nn.Sigmoid()
    for ry, ay, rp, ap, cy, cp in zip(region_y, angle_y, region_p, angle_p, class_y, class_p):
        if cy == 1: 
            doa_y = int(at[int(ry)] + ay[int(ry)].item() * 60)

            rp_idx = int(torch.argmax(rp))
            doa_p = int(at[rp_idx] + sig(ap[rp_idx]) * 60)
            #print(doa_p)
            
            if doa_p > 360:
                doa_p = doa_p - 360
            elif doa_p < 0:
                doa_p = 360 + doa_p

            residual = abs(doa_y - doa_p)
            if residual > 180:
                residual = 360 - residual
            doa_err += residual

            if residual <= threshold:
                bo = 1
            else:
                bo = 0
            doa_acc += bo
            count += 1
        else:
            continue
    return doa_err, doa_acc, count, doa_p

In [13]:
valid_dataloader = torch.utils.data.DataLoader(test_dataset,
                                               batch_size=1, )

# Location of original source
azimuth = 60.0 / 180.0 * np.pi  # 60 degrees
distance = 2.0  # 3 meters
dim = 3  # dimensions (2 or 3)
room_dim = np.r_[7.0, 7.0, 3.0]

# Use AnechoicRoom or ShoeBox implementation. The results are equivalent because max_order=0 for both.
# The plots change a little because in one case there are no walls.
use_anechoic_class = True

print("============ Using anechoic: {} ==================".format(use_anechoic_class))

#######################
# algorithms parameters
SNR = 30.0  # signal-to-noise ratio
c = 343.0  # speed of sound
fs = 16000  # sampling frequency
nfft = 256  # FFT size
freq_bins = np.arange(5, 60)  # FFT bins to use for estimation

algo_names = sorted(pra.doa.algorithms.keys())
algo_names = algo_names[2:6]

R = pra.circular_2D_array(room_dim[:2] / 2, 6, 0.0, 0.04)
model_error = 0.
srp_error = 0.
tops_error = 0.
music_error = 0.
normMusic_error = 0.

srp_acc = 0.
tops_acc = 0.
music_acc = 0.
normMusic_acc = 0.
count = 0 

for batch_index, (x_sg, y_region, y_angle, y_class, bce, angle) in enumerate(valid_dataloader):

    if y_class == 1:
        count += 1
        print(angle, y_class)
        m_angle = angle
        angle = angle * -1 + 120
        if angle < 0 :
            angle = angle + 360

        
        out_localizer1, out_localizer2, out_detector = net(x_sg)
        x_sg = x_sg.numpy()[0]
        X_sg = np.array(
            [
                pra.transform.stft.analysis(signal, nfft, nfft // 2).T
                for signal in x_sg
            ]
        )

        _, region_pidx = out_localizer1.max(1)

        a, b, c, d = DOA(y_region, y_angle, out_localizer1, out_localizer2, 20, y_class, torch.sigmoid(out_detector))
        print(d, m_angle)
        model_error += abs(d - int(m_angle.numpy()[0])) if abs(d - int(m_angle.numpy()[0]))<=180 else 360 - abs(d - int(m_angle.numpy()[0])) 
        print(angle.numpy()[0])
        for algo_name in algo_names:
            # Construct the new DOA object
            # the max_four parameter is necessary for FRIDA only
            doa = pra.doa.algorithms[algo_name](R, fs, nfft, c=c, max_four=4, num_src=3)

            # this call here perform localization on the frames in X
            doa.locate_sources(X_sg, freq_bins=freq_bins)

            # doa.azimuth_recon contains the reconstructed location of the source
            print(algo_name)
            print("  Recovered azimuth:", doa.azimuth_recon / np.pi * 180.0, "degrees")
            value = min(abs(angle.numpy()[0] - doa.azimuth_recon / np.pi * 180.0)) \
                    if min(abs(angle.numpy()[0] - doa.azimuth_recon / np.pi * 180.0)) <= 180 else \
                    360 - min(abs(angle.numpy()[0] - doa.azimuth_recon / np.pi * 180.0))
            
            if algo_name == 'SRP':
                srp_error += value
                if value <= 20:
                    srp_acc += 1
                
            if algo_name == 'MUSIC':
                music_error += value
                if value <= 20:
                    music_acc += 1
                
                
            if algo_name == 'NormMUSIC':
                normMusic_error += value
                if value <= 20:
                    normMusic_acc += 1
                
            if algo_name == 'TOPS':
                tops_error += value
                if value <= 20:
                    tops_acc += 1
            #print("  Error:", circ_dist(azimuth, doa.azimuth_recon) / np.pi * 180.0, "degrees")


============ Using anechoic: True ==================
tensor([265]) tensor([1.])
264 tensor([265])
215
MUSIC
  Recovered azimuth: [ 76. 104. 256.] degrees
NormMUSIC
  Recovered azimuth: [ 97.  30. 210.] degrees
SRP
  Recovered azimuth: [ 37. 210.  30.] degrees
TOPS
  Recovered azimuth: [337. 210.  30.] degrees
tensor([186]) tensor([1.])
186 tensor([186])
294
MUSIC
  Recovered azimuth: [ 16. 284.  76.] degrees
NormMUSIC
  Recovered azimuth: [143.  16.  97.] degrees
SRP
  Recovered azimuth: [150.  90. 270.] degrees
TOPS
  Recovered azimuth: [210.  83. 157.] degrees
tensor([18]) tensor([1.])
16 tensor([18])
102
MUSIC
  Recovered azimuth: [239.  44. 196.] degrees
NormMUSIC
  Recovered azimuth: [ 60.   0. 180.] degrees
SRP
  Recovered azimuth: [330.  30. 157.] degrees
TOPS
  Recovered azimuth: [270.  90. 150.] degrees
tensor([117]) tensor([1.])
199 tensor([117])
3
MUSIC
  Recovered azimuth: [  0. 120. 330.] degrees
NormMUSIC
  Recovered azimuth: [ 23. 270. 143.] degrees
SRP
  Recovered azimu

TOPS
  Recovered azimuth: [323. 270.  90.] degrees
tensor([7]) tensor([1.])
10 tensor([7])
113
MUSIC
  Recovered azimuth: [210. 270.  44.] degrees
NormMUSIC
  Recovered azimuth: [270. 263.  44.] degrees
SRP
  Recovered azimuth: [ 30. 210. 270.] degrees
TOPS
  Recovered azimuth: [ 37. 203. 337.] degrees
tensor([310]) tensor([1.])
307 tensor([310])
170
MUSIC
  Recovered azimuth: [ 60. 180.   0.] degrees
NormMUSIC
  Recovered azimuth: [63.  0. 23.] degrees
SRP
  Recovered azimuth: [ 97. 337.  23.] degrees
TOPS
  Recovered azimuth: [ 97. 337. 263.] degrees
tensor([103]) tensor([1.])
89 tensor([103])
17
MUSIC
  Recovered azimuth: [ 23.  16. 224.] degrees
NormMUSIC
  Recovered azimuth: [330.  97. 270.] degrees
SRP
  Recovered azimuth: [270. 210.  30.] degrees
TOPS
  Recovered azimuth: [270. 150. 330.] degrees
tensor([4]) tensor([1.])
4 tensor([4])
116
MUSIC
  Recovered azimuth: [136.  76.  44.] degrees
NormMUSIC
  Recovered azimuth: [ 60. 300. 203.] degrees
SRP
  Recovered azimuth: [203. 210

TOPS
  Recovered azimuth: [143. 217. 157.] degrees
tensor([250]) tensor([1.])
246 tensor([250])
230
MUSIC
  Recovered azimuth: [ 44. 316. 164.] degrees
NormMUSIC
  Recovered azimuth: [323.  83.  30.] degrees
SRP
  Recovered azimuth: [ 37. 210.  30.] degrees
TOPS
  Recovered azimuth: [ 97. 210.  30.] degrees
tensor([219]) tensor([1.])
229 tensor([219])
261
MUSIC
  Recovered azimuth: [210. 330. 316.] degrees
NormMUSIC
  Recovered azimuth: [ 90. 316. 270.] degrees
SRP
  Recovered azimuth: [ 37. 270.  90.] degrees
TOPS
  Recovered azimuth: [ 90. 270. 210.] degrees
tensor([267]) tensor([1.])
263 tensor([267])
213
MUSIC
  Recovered azimuth: [256.  76. 164.] degrees
NormMUSIC
  Recovered azimuth: [120.  30. 210.] degrees
SRP
  Recovered azimuth: [330. 210.  30.] degrees
TOPS
  Recovered azimuth: [ 83. 210.  30.] degrees
tensor([138]) tensor([1.])
149 tensor([138])
342
MUSIC
  Recovered azimuth: [330. 284. 270.] degrees
NormMUSIC
  Recovered azimuth: [ 48. 330. 284.] degrees
SRP
  Recovered az

TOPS
  Recovered azimuth: [263. 337.  30.] degrees
tensor([154]) tensor([1.])
147 tensor([154])
326
MUSIC
  Recovered azimuth: [270.  30.  90.] degrees
NormMUSIC
  Recovered azimuth: [120. 300. 330.] degrees
SRP
  Recovered azimuth: [ 90. 150. 330.] degrees
TOPS
  Recovered azimuth: [217. 150. 330.] degrees
tensor([90]) tensor([1.])
94 tensor([90])
30
MUSIC
  Recovered azimuth: [316. 256. 164.] degrees
NormMUSIC
  Recovered azimuth: [300. 210.  30.] degrees
SRP
  Recovered azimuth: [323.  30. 210.] degrees
TOPS
  Recovered azimuth: [ 83.  30. 210.] degrees
tensor([212]) tensor([1.])
221 tensor([212])
268
MUSIC
  Recovered azimuth: [118. 316. 164.] degrees
NormMUSIC
  Recovered azimuth: [180.  83. 270.] degrees
SRP
  Recovered azimuth: [ 90.  30. 210.] degrees
TOPS
  Recovered azimuth: [270.  83. 157.] degrees
tensor([181]) tensor([1.])
184 tensor([181])
299
MUSIC
  Recovered azimuth: [224.  16.  44.] degrees
NormMUSIC
  Recovered azimuth: [150. 120.  90.] degrees
SRP
  Recovered azimut

TOPS
  Recovered azimuth: [157.  83. 210.] degrees
tensor([59]) tensor([1.])
63 tensor([59])
61
MUSIC
  Recovered azimuth: [164. 344. 224.] degrees
NormMUSIC
  Recovered azimuth: [120.  23. 217.] degrees
SRP
  Recovered azimuth: [ 90. 143. 217.] degrees
TOPS
  Recovered azimuth: [210. 217.  83.] degrees
tensor([263]) tensor([1.])
253 tensor([263])
217
MUSIC
  Recovered azimuth: [ 30. 164. 256.] degrees
NormMUSIC
  Recovered azimuth: [240. 210.  30.] degrees
SRP
  Recovered azimuth: [270. 210.  30.] degrees
TOPS
  Recovered azimuth: [ 30. 210. 157.] degrees
tensor([128]) tensor([1.])
135 tensor([128])
352
MUSIC
  Recovered azimuth: [150.  90. 136.] degrees
NormMUSIC
  Recovered azimuth: [203.  83. 157.] degrees
SRP
  Recovered azimuth: [ 90. 210. 270.] degrees
TOPS
  Recovered azimuth: [277. 210.  83.] degrees
tensor([162]) tensor([1.])
149 tensor([162])
318
MUSIC
  Recovered azimuth: [ 44.  16. 196.] degrees
NormMUSIC
  Recovered azimuth: [120. 150. 330.] degrees
SRP
  Recovered azimut

...

TOPS
  Recovered azimuth: [330. 210. 150.] degrees
tensor([21]) tensor([1.])
21 tensor([21])
99
MUSIC
  Recovered azimuth: [16. 30. 90.] degrees
NormMUSIC
  Recovered azimuth: [300. 270.  90.] degrees
SRP
  Recovered azimuth: [ 30. 270.  90.] degrees
TOPS
  Recovered azimuth: [323.  90. 270.] degrees
tensor([174]) tensor([1.])
172 tensor([174])
306
MUSIC
  Recovered azimuth: [ 90. 164. 224.] degrees
NormMUSIC
  Recovered azimuth: [ 23.  90. 224.] degrees
SRP
  Recovered azimuth: [ 97. 270.  90.] degrees
TOPS
  Recovered azimuth: [150.  97.  23.] degrees
tensor([256]) tensor([1.])
252 tensor([256])
224
MUSIC
  Recovered azimuth: [120. 164.  90.] degrees
NormMUSIC
  Recovered azimuth: [240. 300. 120.] degrees
SRP
  Recovered azimuth: [157. 210.  30.] degrees
TOPS
  Recovered azimuth: [ 83.  90. 270.] degrees
tensor([112]) tensor([1.])
115 tensor([112])
8
MUSIC
  Recovered azimuth: [270. 330. 256.] degrees
NormMUSIC
  Recovered azimuth: [210. 157. 203.] degrees
SRP
  Recovered azimuth: [2

TOPS
  Recovered azimuth: [ 30.  37. 330.] degrees
tensor([162]) tensor([1.])
156 tensor([162])
318
MUSIC
  Recovered azimuth: [356. 256.  97.] degrees
NormMUSIC
  Recovered azimuth: [263. 337.  97.] degrees
SRP
  Recovered azimuth: [337.  97. 263.] degrees
TOPS
  Recovered azimuth: [150. 337. 263.] degrees
tensor([149]) tensor([1.])
163 tensor([149])
331
MUSIC
  Recovered azimuth: [224.  44.  16.] degrees
NormMUSIC
  Recovered azimuth: [120. 300. 330.] degrees
SRP
  Recovered azimuth: [203. 330. 150.] degrees
TOPS
  Recovered azimuth: [203. 150. 330.] degrees
tensor([90]) tensor([1.])
87 tensor([90])
30
MUSIC
  Recovered azimuth: [120. 300.  76.] degrees
NormMUSIC
  Recovered azimuth: [300.  30. 210.] degrees
SRP
  Recovered azimuth: [263.  30. 210.] degrees
TOPS
  Recovered azimuth: [337. 210.  30.] degrees
tensor([72]) tensor([1.])
76 tensor([72])
48
MUSIC
  Recovered azimuth: [ 90. 164. 344.] degrees
NormMUSIC
  Recovered azimuth: [164. 210.  30.] degrees
SRP
  Recovered azimuth: [

TOPS
  Recovered azimuth: [330. 143. 150.] degrees
tensor([266]) tensor([1.])
261 tensor([266])
214
MUSIC
  Recovered azimuth: [  0. 300. 120.] degrees
NormMUSIC
  Recovered azimuth: [256.  30. 210.] degrees
SRP
  Recovered azimuth: [263. 210.  30.] degrees
TOPS
  Recovered azimuth: [157. 210.  30.] degrees
tensor([258]) tensor([1.])
258 tensor([258])
222
MUSIC
  Recovered azimuth: [ 30. 164. 210.] degrees
NormMUSIC
  Recovered azimuth: [  0. 300. 120.] degrees
SRP
  Recovered azimuth: [210.  90. 270.] degrees
TOPS
  Recovered azimuth: [150.  90. 270.] degrees
tensor([47]) tensor([1.])
39 tensor([47])
73
MUSIC
  Recovered azimuth: [210.  30. 270.] degrees
NormMUSIC
  Recovered azimuth: [  0.  60. 240.] degrees
SRP
  Recovered azimuth: [217.  90. 270.] degrees
TOPS
  Recovered azimuth: [143. 270. 217.] degrees
tensor([214]) tensor([1.])
205 tensor([214])
266
MUSIC
  Recovered azimuth: [316. 157. 191.] degrees
NormMUSIC
  Recovered azimuth: [ 90. 270.  83.] degrees
SRP
  Recovered azimut

TOPS
  Recovered azimuth: [210.  30. 157.] degrees
tensor([1]) tensor([1.])
11 tensor([1])
119
MUSIC
  Recovered azimuth: [ 16. 358.  44.] degrees
NormMUSIC
  Recovered azimuth: [240. 300. 120.] degrees
SRP
  Recovered azimuth: [323.  90. 270.] degrees
TOPS
  Recovered azimuth: [ 37.  90. 270.] degrees
tensor([300]) tensor([1.])
318 tensor([300])
180
MUSIC
  Recovered azimuth: [104. 256. 284.] degrees
NormMUSIC
  Recovered azimuth: [256. 150.  37.] degrees
SRP
  Recovered azimuth: [210. 330. 150.] degrees
TOPS
  Recovered azimuth: [330.  97.  23.] degrees
tensor([61]) tensor([1.])
54 tensor([61])
59
MUSIC
  Recovered azimuth: [337. 136. 236.] degrees
NormMUSIC
  Recovered azimuth: [270. 217. 263.] degrees
SRP
  Recovered azimuth: [217. 337. 263.] degrees
TOPS
  Recovered azimuth: [ 30.  90. 330.] degrees
tensor([335]) tensor([1.])
345 tensor([335])
145
MUSIC
  Recovered azimuth: [330.  30. 210.] degrees
NormMUSIC
  Recovered azimuth: [330.  90. 240.] degrees
SRP
  Recovered azimuth: [2

TOPS
  Recovered azimuth: [143.  90. 270.] degrees
tensor([314]) tensor([1.])
327 tensor([314])
166
MUSIC
  Recovered azimuth: [240. 180.   0.] degrees
NormMUSIC
  Recovered azimuth: [  0. 330. 210.] degrees
SRP
  Recovered azimuth: [150.  30. 330.] degrees
TOPS
  Recovered azimuth: [150.  30. 210.] degrees
tensor([180]) tensor([1.])
182 tensor([180])
300
MUSIC
  Recovered azimuth: [ 30. 224.  90.] degrees
NormMUSIC
  Recovered azimuth: [ 60. 120. 300.] degrees
SRP
  Recovered azimuth: [ 37. 330. 203.] degrees
TOPS
  Recovered azimuth: [ 37.  90. 270.] degrees
tensor([175]) tensor([1.])
185 tensor([175])
305
MUSIC
  Recovered azimuth: [131.  16. 224.] degrees
NormMUSIC
  Recovered azimuth: [157.  60.  83.] degrees
SRP
  Recovered azimuth: [270.  90.  30.] degrees
TOPS
  Recovered azimuth: [ 30.  83. 157.] degrees
tensor([182]) tensor([1.])
201 tensor([182])
298
MUSIC
  Recovered azimuth: [300. 120. 224.] degrees
NormMUSIC
  Recovered azimuth: [300. 323. 150.] degrees
SRP
  Recovered az

In [12]:
srp_error / count, model_error / count, tops_error / count, normMusic_error / count, music_error / count

(33.551206010280744,
 12.408857255832345,
 44.92447607750099,
 39.688612099644125,
 49.94800316330566)

In [14]:
srp_acc / count, model_error / count, tops_acc / count, normMusic_acc / count, music_acc / count

(0.5179913009094503,
 12.408857255832345,
 0.42744167655199683,
 0.5215500197706603,
 0.24930802688809806)